In [ ]:
from deap import base, creator, tools, algorithms
import random
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
ONE_MAX_LENGTH = 100

POPULATION_SIZE = 200
P_CROSSOVER = 0.9
P_MUTATION = 0.1
MAX_GENERATIONS = 50
HALL_OF_FAME_SIZE = 10

In [ ]:
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

In [ ]:
toolbox.register("zero_or_one", random.randint, 0, 1)

In [ ]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))

In [ ]:
creator.create("Individual", list, fitness=creator.FitnessMax)

In [ ]:
toolbox.register("individual_creator", tools.initRepeat, creator.Individual, toolbox.zero_or_one, ONE_MAX_LENGTH)

In [ ]:
toolbox.register("population_creator", tools.initRepeat, list, toolbox.individual_creator)

In [ ]:
def one_max_fitness(individual):
    return sum(individual),

In [ ]:
toolbox.register("evaluate", one_max_fitness)

In [ ]:
toolbox.register("select", tools.selRoulette)
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=1/ONE_MAX_LENGTH)

In [ ]:
population = toolbox.population_creator(n=POPULATION_SIZE)
generation_counter = 0

In [ ]:
fitness_values = list(map(toolbox.evaluate, population))

In [ ]:
for individual, fitness_value in zip(population, fitness_values):
    individual.fitness.values = fitness_value

In [ ]:
fitness_values = [individual.fitness.values[0] for individual in population]

In [ ]:
max_fitness_values = []
mean_fitness_values = []

In [ ]:
while max(fitness_values) < ONE_MAX_LENGTH and generation_counter < MAX_GENERATIONS:
    generation_counter += 1
    offspring = toolbox.select(population, len(population))
    offspring = list(map(toolbox.clone, offspring))
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < P_CROSSOVER:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values
    
    for mutant in offspring:
        if random.random() < P_MUTATION:
            toolbox.mutate(mutant)
            del mutant.fitness.values
    
    fresh_individuals = [ind for ind in offspring if not ind.fitness.valid]
    fresh_fitness_values = list(map(toolbox.evaluate, fresh_individuals))
    for individual, fitness_value in zip(fresh_individuals, fresh_fitness_values):
        individual.fitness.values = fitness_value
        
    population[:] = offspring
    fitness_values = [ind.fitness.values[0] for ind in population]
    
    max_fitness = max(fitness_values)
    mean_fitness = sum(fitness_values) / len(fitness_values)
    max_fitness_values.append(max_fitness)
    mean_fitness_values.append(mean_fitness)
    print(f"- Generation {generation_counter}: Max Fitness = {max_fitness}, Avg Fitness = {mean_fitness}")
    
    best_index = fitness_values.index(max(fitness_values))
    print("Best Individual", *population[best_index], "\n")

In [ ]:
plt.plot(max_fitness_values, color='red')
plt.plot(mean_fitness_values, color='green')
plt.xlabel('Generation')
plt.ylabel('Max / Average Fitness')
plt.title('Max and Average Fitness over Generations')
plt.show()

In [ ]:
stats = tools.Statistics(lambda ind: ind.fitness.values)

In [ ]:
stats.register("max", np.max)
stats.register("avg", np.mean)

In [ ]:
hof = tools.HallOfFame(HALL_OF_FAME_SIZE)
population = toolbox.population_creator(n=POPULATION_SIZE)
population, logbook = algorithms.eaSimple(population, toolbox, cxpb=P_CROSSOVER, 
                                          mutpb=P_MUTATION, ngen=MAX_GENERATIONS,
                                          halloffame=hof, stats=stats, verbose=True)

In [ ]:
max_fitness_values, mean_fitness_values = logbook.select("max", "avg")

In [ ]:
plt.plot(max_fitness_values, color='red')
plt.plot(mean_fitness_values, color='green')
plt.xlabel('Generation')
plt.ylabel('Max / Average Fitness')
plt.title('Max and Average Fitness over Generations')
plt.show()

In [ ]:
print("Hall of Fame Indivuals = ", *hof.items, sep="\n")
print("Best Ever Individual = ", hof.items[0])